<a href="https://colab.research.google.com/github/RuiHu0505/inf1340-programmingfordatascience-fa22/blob/main/RuiHu/Midterm_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**0. Tidy Data Principal**


1. Column names need to be informative, variable names and not values
2. Each column needs to consist of one and only one variable
3. Variables need to be in cells, not rows and columns
4. Each table column needs to have a singular data type
5. A single observational units must be in one table




**1. Import Data from Excel File**


> Separately import different tab

> Easy to control header location







In [ ]:
import pandas as pd
import numpy as np
import copy

# import data from excel, skip top of the table and import multi-index headers
df_content = pd.read_excel('UN_MigrantStockTotal_2015.xlsx',sheet_name = 'CONTENTS', skiprows = 14, header=[1])
df_1_ori = pd.read_excel('UN_MigrantStockTotal_2015.xlsx',sheet_name = 'Table 1', skiprows = 13, header=[1,2])
df_2_ori = pd.read_excel('UN_MigrantStockTotal_2015.xlsx',sheet_name = 'Table 2', skiprows = 13, header=[1,2])
df_3_ori = pd.read_excel('UN_MigrantStockTotal_2015.xlsx',sheet_name = 'Table 3', skiprows = 13, header=[1,2])
df_4_ori = pd.read_excel('UN_MigrantStockTotal_2015.xlsx',sheet_name = 'Table 4', skiprows = 13, header=[1,2])
df_5_ori = pd.read_excel('UN_MigrantStockTotal_2015.xlsx',sheet_name = 'Table 5', skiprows = 13, header=[1,2])
df_6_ori = pd.read_excel('UN_MigrantStockTotal_2015.xlsx',sheet_name = 'Table 6', skiprows = 13, header=[1,2])

df_country = pd.read_excel('UN_MigrantStockTotal_2015.xlsx',sheet_name='ANNEX', skiprows = 13, header = [1])
df_notes = pd.read_excel('UN_MigrantStockTotal_2015.xlsx',sheet_name='NOTES', skiprows = 13, header = [1])


**Sample Input DataFrame**

In [ ]:
df_1_ori.head(10)

Sort\norder Major area, region, country or area of destination  \
  Unnamed: 0_level_1                                 Unnamed: 1_level_1   
0                  1                                              WORLD   
1                  2                                  Developed regions   
2                  3                                 Developing regions   
3                  4                          Least developed countries   
4                  5  Less developed regions excluding least develop...   
5                  6                                 Sub-Saharan Africa   
6                  7                                             Africa   
7                  8                                     Eastern Africa   
8                  9                                            Burundi   
9                 10                                            Comoros   

               Notes       Country code   Type of data (a)  \
  Unnamed: 2_level_1 Unnamed: 3_level_1 Unnamed: 4_level_1   
0                NaN                900                NaN   
1                (b)                901                NaN   
2                (c)                902                NaN   
3                (d)                941                NaN   
4                NaN                934                NaN   
5                (e)                947                NaN   
6                NaN                903                NaN   
7                NaN                910                NaN   
8                NaN                108                B R   
9                NaN                174                  B   

  International migrant stock at mid-year (both sexes)                        \
                                                  1990       1995       2000   
0                                          152563212    160801752  172703309   
1                                           82378628     92306854  103375363   
2                                           70184584     68494898   69327946   
3                                           11075966     11711703   10077824   
4                                           59105261     56778501   59244124   
5                                           14690319     15324570   13716539   
6                                           15690623     16352814   14800306   
7                                            5964031      5022742    4844795   
8                                             333110       254853     125628   
9                                              14079        13939      13799   

                         ... International migrant stock at mid-year (male)  \
        2005       2010  ...                                           2000   
0  191269100  221714243  ...                                       87884839   
1  117181109  132560325  ...                                       50536796   
2   74087991   89153918  ...                                       37348043   
3    9809634   10018128  ...                                        5361902   
4   64272611   79130668  ...                                       31986141   
5   13951086   15496764  ...                                        7210452   
6   15191146   16840014  ...                                        7856358   
7    4745792    4657063  ...                                        2480584   
8     172874     235259  ...                                          61094   
9      13209      12618  ...                                           6511   

                                   \
       2005       2010       2015   
0  97866674  114613714  126115435   
1  57217777   64081077   67618619   
2  40648897   50532637   58496816   
3   5383009    5462714    6463217   
4  35265888   45069923   52033599   
5   7444048    8188581   10099486   
6   8231437    9039314   11123423   
7   2529460    2366216    3109176   
8     84805     115823     141311   
9      6286       6060       6071   

  Intern

**2. Function used for Data Cleaning**


> Most of dataframes have similar structures, thus requires similar operations for data cleaning => one generic function **melt_df**
*   Input: DataFrame, DataFrame, String
*   Output: DataFrame, DataFrame

> Table 6 contains three type of data fields, thus needs extra steps => **melt6_df**
*   Input: DataFrame, DataFrame
*   Output: DataFrame, DataFrame

> All tables needs separation of country and major areas data => **split_df**
*   Input: DataFrame, DataFrame
*   Output: DataFrame, DataFrame





In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')


# function for general cleaning for table 1,2,3,5
def melt_df(df, df_country, name):
  # check id_vars and value_vars
  id_vars = [idv for idv in df.columns if 'Unnamed' in str(idv[1])]
  value_vars = [valv for valv in df.columns if 'Unnamed' not in str(valv[1])]
  # df.melt, multi-level
  df_multiidx = df.melt(id_vars=id_vars[1:], value_vars=value_vars,var_name=['Gender','Year'], value_name = name)
  # rename df
  df_multiidx = df_multiidx.rename(columns = {col : col[0] for col in id_vars})
  # replace missing value with NaN
  df_multiidx= df_multiidx.replace('..', np.nan)
  # replace gender information
  df_multiidx.loc[df_multiidx.Gender.str.contains("both sexes"),'Gender'] = "Both"
  df_multiidx.loc[df_multiidx.Gender.str.contains("female"),'Gender'] = "F"
  df_multiidx.loc[df_multiidx.Gender.str.contains("male"),'Gender'] = "M"

  return split_df(df_multiidx,df_country)

# specific function for table 6 => three columns of values instead of one
def melt6_df(df,df_country):
  # check id_vars and value_vars
  id_vars = [idv for idv in df.columns if 'Unnamed' in str(idv[1])]
  value_vars1 = [valv for valv in df.columns if ('Unnamed' not in str(valv[1])) and ('-' not in str(valv[1]))]
  value_vars2 = [valv for valv in df.columns if ('Unnamed' not in str(valv[1])) and ('-' in str(valv[1]))]
  # df.melt, multi-level, then pivot back to multi-column values
  df_1 = df.melt(id_vars= id_vars[1:], value_vars=value_vars1,var_name=['Type','Year'], value_name = 'value1').pivot(index = id_vars[1:] + ['Year'], columns = ['Type'], values = 'value1').reset_index()
  df_2 = df.melt(id_vars= id_vars[1:], value_vars=value_vars2,var_name=['Type','Year'], value_name = 'value1').pivot(index = id_vars[1:] + ['Year'], columns = ['Type'], values = 'value1').reset_index()
  # rename df
  df_1,df_2 = df_1.rename(columns = {col : col[0] for col in id_vars}), df_2.rename(columns = {col : col[0] for col in id_vars})
  # replace missing value with NaN
  df_1,df_2= df_1.replace('..', np.nan), df_2.replace('..',np.nan)

  df_a, df_b = split_df(df_1,df_country)
  df_c, df_d = split_df(df_2,df_country)
  return df_a, df_b, df_c, df_d

# split area into country and major area => based on ANNEX table
def split_df(df,df_country):
  df["isCountry"] = [i in list(df_country.iloc[:,1]) for i in df.iloc[:,0]]
  df_c = df[df["isCountry"] == True].rename(columns = {'Major area, region, country or area of destination':'Country'})
  df_a = df[df["isCountry"] == False].rename(columns = {'Major area, region, country or area of destination':'Major Area'})
  df_c, df_a = df_c.drop("isCountry",axis = 1), df_a.drop("isCountry",axis = 1)
  return df_c, df_a

**3. Output DataFrame**

In [ ]:
df_1_c, df_1_a = melt_df(df_1_ori, df_country, "International migrant stock at mid-year")
df_2_c, df_2_a= melt_df(df_2_ori, df_country, "Total population at mid-year (thousands)")
df_3_c, df_3_a = melt_df(df_3_ori, df_country, "International migrant stock as a percentage of the total population")
df_4_c, df_4_a= melt_df(df_4_ori, df_country, "Female migrants as a percentage of the international migrant stock")
df_5_c, df_5_a = melt_df(df_5_ori, df_country, "Annual rate of change of the migrant stock")
df_6a_c, df_6a_a, df_6b_c, df_6b_a = melt6_df(df_6_ori, df_country)
# drop gender columns for table 4
df_4_c, df_4_a = df_4_c.drop('Gender',axis = 1), df_4_a.drop('Gender',axis = 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resulting Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.
  # This is added back by InteractiveShellApp.init_path()


**4. Format Data Field for demonstration**

In [ ]:
# use to format value column, "N" for whole number and "P" for percentage
def format_df(df,ty):
  name = df.columns[-1]
  df1 = copy.deepcopy(df)
  if ty == "N": df1[name] = df1[name].map('{:,.0f}'.format)
  elif ty == "P": df1[name] = df1[name].map('{:.2f}%'.format)
  elif ty == "S": 
    df1[name] = df1[name].map('{:.2f}%'.format)
    df1[df1.columns[-2]] = df1[df1.columns[-2]].map('{:,.0f}'.format)
  return df1

**Table of Contents**

In [ ]:
df_content

,TABLE,TITLE
0,Table 1,International migrant stock at mid-year by sex...
1,Table 2,Total population at mid-year by sex and by maj...
2,Table 3,International migrant stock as a percentage of...
3,Table 4,Female migrants as a percentage of the interna...
4,Table 5,Annual rate of change of the migrant stock by ...
5,Table 6,Estimated refugee stock at mid-year by major a...
6,ANNEX,Classification of countries and areas by major...
7,NOTES,NOTES


**Table 1: 	International migrant stock at mid-year by sex and by major area, region, country or area, 1990-2015**

In [ ]:
format_df(df_1_c,"N").head(10)

,Country,Notes,Country code,Type of data (a),Gender,Year,International migrant stock at mid-year
8,Burundi,NaN,108,B R,Both,1990,"333,110"
9,Comoros,NaN,174,B,Both,1990,"14,079"
10,Djibouti,NaN,262,B R,Both,1990,"122,221"
11,Eritrea,NaN,232,I,Both,1990,"11,848"
12,Ethiopia,NaN,231,B R,Both,1990,"1,155,390"
13,Kenya,NaN,404,B R,Both,1990,"297,292"
14,Madagascar,NaN,450,C,Both,1990,"23,917"
15,Malawi,NaN,454,B R,Both,1990,"1,127,724"
16,Mauritius,(1),480,C,Both,1990,"3,613"
17,Mayotte,NaN,175,B,Both,1990,"15,229"


In [ ]:
format_df(df_1_a,"N").head(10)

,Major Area,Notes,Country code,Type of data (a),Gender,Year,International migrant stock at mid-year
0,WORLD,NaN,900,NaN,Both,1990,"152,563,212"
1,Developed regions,(b),901,NaN,Both,1990,"82,378,628"
2,Developing regions,(c),902,NaN,Both,1990,"70,184,584"
3,Least developed countries,(d),941,NaN,Both,1990,"11,075,966"
4,Less developed regions excluding least develop...,NaN,934,NaN,Both,1990,"59,105,261"
5,Sub-Saharan Africa,(e),947,NaN,Both,1990,"14,690,319"
6,Africa,NaN,903,NaN,Both,1990,"15,690,623"
7,Eastern Africa,NaN,910,NaN,Both,1990,"5,964,031"
28,Middle Africa,NaN,911,NaN,Both,1990,"1,460,530"
38,Northern Africa,NaN,912,NaN,Both,1990,"2,403,200"


**Table 2: Total population at mid-year by sex and by major area, region, country or area, 1990-2015 (thousands)**

In [ ]:
format_df(df_2_c,"N").head(10)

,Country,Notes,Country code,Gender,Year,Total population at mid-year (thousands)
8,Burundi,NaN,108,Both,1990,"5,613"
9,Comoros,NaN,174,Both,1990,415
10,Djibouti,NaN,262,Both,1990,588
11,Eritrea,NaN,232,Both,1990,"3,139"
12,Ethiopia,NaN,231,Both,1990,"48,057"
13,Kenya,NaN,404,Both,1990,"23,446"
14,Madagascar,NaN,450,Both,1990,"11,546"
15,Malawi,NaN,454,Both,1990,"9,409"
16,Mauritius,(1),480,Both,1990,"1,056"
17,Mayotte,NaN,175,Both,1990,95


In [ ]:

format_df(df_2_a,"N").head(10)

,Major Area,Notes,Country code,Gender,Year,Total population at mid-year (thousands)
0,WORLD,NaN,900,Both,1990,"5,309,668"
1,Developed regions,(b),901,Both,1990,"1,144,463"
2,Developing regions,(c),902,Both,1990,"4,165,205"
3,Least developed countries,(d),941,Both,1990,"510,058"
4,Less developed regions excluding least develop...,NaN,934,Both,1990,"3,655,147"
5,Sub-Saharan Africa,(e),947,Both,1990,"491,498"
6,Africa,NaN,903,Both,1990,"631,614"
7,Eastern Africa,NaN,910,Both,1990,"198,232"
28,Middle Africa,NaN,911,Both,1990,"70,886"
38,Northern Africa,NaN,912,Both,1990,"140,117"


**Table 3:	International migrant stock as a percentage of the total population, 1990-2015**

In [ ]:
format_df(df_3_c,"P").head(10)

,Country,Notes,Country code,Type of data (a),Gender,Year,International migrant stock as a percentage of the total population
8,Burundi,NaN,108,B R,Both,1990,5.93%
9,Comoros,NaN,174,B,Both,1990,3.39%
10,Djibouti,NaN,262,B R,Both,1990,20.77%
11,Eritrea,NaN,232,I,Both,1990,0.38%
12,Ethiopia,NaN,231,B R,Both,1990,2.40%
13,Kenya,NaN,404,B R,Both,1990,1.27%
14,Madagascar,NaN,450,C,Both,1990,0.21%
15,Malawi,NaN,454,B R,Both,1990,11.99%
16,Mauritius,(1),480,C,Both,1990,0.34%
17,Mayotte,NaN,175,B,Both,1990,16.07%


In [ ]:
format_df(df_3_a,"P").head(10)

,Major Area,Notes,Country code,Type of data (a),Gender,Year,International migrant stock as a percentage of the total population
0,WORLD,NaN,900,NaN,Both,1990,2.87%
1,Developed regions,(b),901,NaN,Both,1990,7.20%
2,Developing regions,(c),902,NaN,Both,1990,1.69%
3,Least developed countries,(d),941,NaN,Both,1990,2.17%
4,Less developed regions excluding least develop...,NaN,934,NaN,Both,1990,1.62%
5,Sub-Saharan Africa,(e),947,NaN,Both,1990,2.99%
6,Africa,NaN,903,NaN,Both,1990,2.48%
7,Eastern Africa,NaN,910,NaN,Both,1990,3.01%
28,Middle Africa,NaN,911,NaN,Both,1990,2.06%
38,Northern Africa,NaN,912,NaN,Both,1990,1.72%


**Table 4: 	Female migrants as a percentage of the international migrant stock by major area, region, country or area, 1990-2015**

In [ ]:
format_df(df_4_c,"P").head(10)

,Country,Notes,Country code,Type of data (a),Year,Female migrants as a percentage of the international migrant stock
8,Burundi,NaN,108,B R,1990,50.99%
9,Comoros,NaN,174,B,1990,52.29%
10,Djibouti,NaN,262,B R,1990,47.44%
11,Eritrea,NaN,232,I,1990,47.43%
12,Ethiopia,NaN,231,B R,1990,47.44%
13,Kenya,NaN,404,B R,1990,45.89%
14,Madagascar,NaN,450,C,1990,44.19%
15,Malawi,NaN,454,B R,1990,51.54%
16,Mauritius,(1),480,C,1990,51.20%
17,Mayotte,NaN,175,B,1990,42.35%


In [ ]:
format_df(df_4_a,"P").head(10)

,Major Area,Notes,Country code,Type of data (a),Year,Female migrants as a percentage of the international migrant stock
0,WORLD,NaN,900,NaN,1990,49.04%
1,Developed regions,(b),901,NaN,1990,51.12%
2,Developing regions,(c),902,NaN,1990,46.59%
3,Least developed countries,(d),941,NaN,1990,47.26%
4,Less developed regions excluding least develop...,NaN,934,NaN,1990,46.47%
5,Sub-Saharan Africa,(e),947,NaN,1990,47.28%
6,Africa,NaN,903,NaN,1990,47.23%
7,Eastern Africa,NaN,910,NaN,1990,48.50%
28,Middle Africa,NaN,911,NaN,1990,49.03%
38,Northern Africa,NaN,912,NaN,1990,48.79%


**Table 5: 	Annual rate of change of the migrant stock by sex and by major area, region, country or area, 1990-2015 (percentage)**

In [ ]:
format_df(df_5_c,"P").head(10)

,Country,Notes,Country code,Type of data (a),Gender,Year,Annual rate of change of the migrant stock
8,Burundi,NaN,108,B R,Both,1990-1995,-5.36%
9,Comoros,NaN,174,B,Both,1990-1995,-0.20%
10,Djibouti,NaN,262,B R,Both,1990-1995,-4.06%
11,Eritrea,NaN,232,I,Both,1990-1995,0.91%
12,Ethiopia,NaN,231,B R,Both,1990-1995,-7.18%
13,Kenya,NaN,404,B R,Both,1990-1995,14.66%
14,Madagascar,NaN,450,C,Both,1990-1995,-2.43%
15,Malawi,NaN,454,B R,Both,1990-1995,-30.81%
16,Mauritius,(1),480,C,Both,1990-1995,14.59%
17,Mayotte,NaN,175,B,Both,1990-1995,10.94%


In [ ]:
format_df(df_5_a,"P").head(10)

,Major Area,Notes,Country code,Type of data (a),Gender,Year,Annual rate of change of the migrant stock
0,WORLD,NaN,900,NaN,Both,1990-1995,1.05%
1,Developed regions,(b),901,NaN,Both,1990-1995,2.28%
2,Developing regions,(c),902,NaN,Both,1990-1995,-0.49%
3,Least developed countries,(d),941,NaN,Both,1990-1995,1.12%
4,Less developed regions excluding least develop...,NaN,934,NaN,Both,1990-1995,-0.80%
5,Sub-Saharan Africa,(e),947,NaN,Both,1990-1995,0.85%
6,Africa,NaN,903,NaN,Both,1990-1995,0.83%
7,Eastern Africa,NaN,910,NaN,Both,1990-1995,-3.44%
28,Middle Africa,NaN,911,NaN,Both,1990-1995,11.89%
38,Northern Africa,NaN,912,NaN,Both,1990-1995,-2.87%


**Table 6a: Estimated refugee stock at mid-year by major area, region, country or area, 1990-2015**

In [ ]:
format_df(df_6a_c,"S").head(10)

Type,Country,Notes,Country code,Type of data (a),Year,Estimated refugee stock at mid-year (both sexes),Refugees as a percentage of the international migrant stock
0,Afghanistan,NaN,4,B,1990,25,0.04%
1,Afghanistan,NaN,4,B,1995,"19,605",27.41%
2,Afghanistan,NaN,4,B,2000,0,0.00%
3,Afghanistan,NaN,4,B,2005,32,0.04%
4,Afghanistan,NaN,4,B,2010,"6,434",6.29%
5,Afghanistan,NaN,4,B,2015,"300,569",78.61%
12,Albania,NaN,8,B,1990,"3,000",4.54%
13,Albania,NaN,8,B,1995,"4,720",6.61%
14,Albania,NaN,8,B,2000,528,0.69%
15,Albania,NaN,8,B,2005,91,0.14%


In [ ]:
format_df(df_6a_a,"S").head(10)

Type,Major Area,Notes,Country code,Type of data (a),Year,Estimated refugee stock at mid-year (both sexes),Refugees as a percentage of the international migrant stock
6,Africa,NaN,903,NaN,1990,"5,687,352",36.25%
7,Africa,NaN,903,NaN,1995,"5,949,953",36.38%
8,Africa,NaN,903,NaN,2000,"3,609,138",24.39%
9,Africa,NaN,903,NaN,2005,"2,750,644",18.11%
10,Africa,NaN,903,NaN,2010,"2,413,917",14.33%
11,Africa,NaN,903,NaN,2015,"4,022,363",19.48%
72,Asia,NaN,935,NaN,1990,"9,937,007",20.64%
73,Asia,NaN,935,NaN,1995,"7,937,682",17.05%
74,Asia,NaN,935,NaN,2000,"9,175,210",18.60%
75,Asia,NaN,935,NaN,2005,"8,109,615",15.19%


**Table 6b: Annual rate of change of the refugee stock**

In [ ]:
format_df(df_6b_c,"P").head(10)

Type,Country,Notes,Country code,Type of data (a),Year,Annual rate of change of the refugee stock
0,Afghanistan,NaN,4,B,1990-1995,128.99%
1,Afghanistan,NaN,4,B,1995-2000,nan%
2,Afghanistan,NaN,4,B,2000-2005,nan%
3,Afghanistan,NaN,4,B,2005-2010,102.91%
4,Afghanistan,NaN,4,B,2010-2015,50.50%
10,Albania,NaN,8,B,1990-1995,7.51%
11,Albania,NaN,8,B,1995-2000,-45.25%
12,Albania,NaN,8,B,2000-2005,-31.78%
13,Albania,NaN,8,B,2005-2010,5.77%
14,Albania,NaN,8,B,2010-2015,85.68%


In [ ]:
format_df(df_6b_a,"P").head(10)

Type,Major Area,Notes,Country code,Type of data (a),Year,Annual rate of change of the refugee stock
5,Africa,NaN,903,NaN,1990-1995,0.08%
6,Africa,NaN,903,NaN,1995-2000,-8.00%
7,Africa,NaN,903,NaN,2000-2005,-5.95%
8,Africa,NaN,903,NaN,2005-2010,-4.67%
9,Africa,NaN,903,NaN,2010-2015,6.13%
60,Asia,NaN,935,NaN,1990-1995,-3.82%
61,Asia,NaN,935,NaN,1995-2000,1.73%
62,Asia,NaN,935,NaN,2000-2005,-4.04%
63,Asia,NaN,935,NaN,2005-2010,1.01%
64,Asia,NaN,935,NaN,2010-2015,1.93%


**Appendix: 	Classification of countries and areas by major area and region**

In [ ]:
df_country.head(10)

,Country code,Country or area,Sort order,Major area,Code,Sort order.1,Region,Code.1,Sort order.2,Developed region,Least developed country,Sub-Saharan Africa
0,4,Afghanistan,99,Asia,935,71,Southern Asia,5501,98,No,Yes,No
1,8,Albania,154,Europe,908,127,Southern Europe,925,153,Yes,No,No
2,12,Algeria,40,Africa,903,7,Northern Africa,912,39,No,No,No
3,16,American Samoa,257,Oceania,909,238,Polynesia,957,256,No,No,No
4,20,Andorra,155,Europe,908,127,Southern Europe,925,153,Yes,No,No
5,24,Angola,30,Africa,903,7,Middle Africa,911,29,No,Yes,Yes
6,660,Anguilla,182,Latin America and the Caribbean,904,180,Caribbean,915,181,No,No,No
7,28,Antigua and Barbuda,183,Latin America and the Caribbean,904,180,Caribbean,915,181,No,No,No
8,32,Argentina,218,Latin America and the Caribbean,904,180,South America,931,217,No,No,No
9,51,Armenia,109,Asia,935,71,Western Asia,922,108,No,No,No


**Appendix: NOTES**

In [ ]:
df_notes.columns = ["FootNotes", "NOTES"]
df_notes.head(10)

,FootNotes,NOTES
0,(a),The column labeled “Type of data” indicates wh...
1,(b),"More developed regions comprise Europe, Northe..."
2,(c),Less developed regions comprise all regions of...
3,(d),"The least developed countries, as defined by t..."
4,(e),Sub-Saharan Africa refers to all of Africa exc...
5,(1),"Including Agalega, Rodrigues and Saint Brandon."
6,(2),Including Zanzibar.
7,(3),The estimates for 1990 to 2005 refer to Sudan ...
8,(4),Including Ascension and Tristan da Cunha.
9,(5),"For statistical purposes, the data for China d..."
